In [1]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [2]:
LOCAL = True

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [3]:
%%capture pwd
!pwd

In [4]:
# uid = "2021-05-05-which-firms-and-industries-have-been-most-affected-by-covid-update"  # article unique ID
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('2021-06-15-how-is-covid-19-catalysing-changes-in-digital-health', 300, 500)

# Fig 1

In [6]:
df = pd.read_excel("raw/Digital Health.xlsx").dropna(how="all")

In [7]:
f = "fig1_ehealth"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
df.head()

,Year,Number of deals,Amount raised (£ million)
0,2011,7,12.20
1,2012,11,4.69
2,2013,13,8.88
3,2014,30,23.70
4,2015,29,26.90


In [92]:
base = alt.Chart(f1).encode(
    x=alt.X(
        "Year:O",
        axis=alt.Axis(
            grid=False,
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            labelAngle=0,
        ),
    )
)
line = base.mark_line(color=colors["eco-background"]).encode(
    y=alt.Y("Number of deals:Q", axis=None)
)
bar = base.mark_bar(color=colors["eco-turquiose"], opacity=0.8, width=35).encode(
    y=alt.Y("Amount raised (£ million):Q", axis=None)
)
btext = (
    bar.mark_text(color=colors["eco-turquiose"], dy=-10)
    .encode(text="l:N")
    .transform_calculate(l="'£'+datum['Amount raised (£ million)']+'M'")
)
ptext = line.mark_text(color=colors["eco-background"], dy=-13, dx=-4).encode(
    text="Number of deals:N"
)
point = base.mark_point(
    color=colors["eco-background"], fill=colors["eco-background"], opacity=1
).encode(y=alt.Y("Number of deals:Q", axis=None))
labels = (
    alt.Chart(
        pd.DataFrame(
            [
                {
                    "x": 2015,
                    "y": 1,
                    "l": "Number of deals",
                    "c": colors["eco-background"],
                },
                {
                    "x": 2017,
                    "y": 0.55,
                    "l": "Amount raised (£ million)",
                    "c": colors["eco-turquiose"],
                },
            ]
        )
    )
    .mark_text(dx=10, dy=-10)
    .encode(
        x="x:O",
        y=alt.Y("y:Q", axis=None, scale=alt.Scale(domain=[0, 1.4])),
        text="l:N",
        color=alt.Color("c:N", scale=None, legend=None),
    )
)
layer = (
    (bar + line + point + btext + ptext + labels)
    .resolve_scale(y="independent")
    .configure_view(stroke=None)
    .properties(title="")
    .properties(height=300, width=500)
)
layer.save("visualisation/" + f + ".json")
layer

alt.LayerChart(...)